In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import re
import string
from collections import Counter
import nltk

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

import gensim

%matplotlib inline

In [2]:
df = pd.read_csv('df1csv.csv')
df.head()

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
0,OPERATIONAL,"{'lat': 20.6939009, 'lng': -105.2742384}",https://maps.gstatic.com/mapfiles/place_api/ic...,casa LAS PIÑAS,{'open_now': False},"[{'height': 640, 'html_attributions': ['<a hre...",ChIJIeVYvYNGIYQRh-9bPVK3fCU,"{'compound_code': 'MPVG+H8 Jarretaderas, Nayar...",4.6,ChIJIeVYvYNGIYQRh-9bPVK3fCU,...,casa LAS PIÑAS,4.6,"[{'author_name': 'Jovita Garcia Fernandez', 'a...",8.0,https://www.facebook.com/departamentos.laspinas/,"[5, 5, 5, 5, 5]","['An excellent place has all the comforts', 'I...",NaN,20.693901,-105.274238
1,OPERATIONAL,"{'lat': 20.6920731, 'lng': -105.2747303}",https://maps.gstatic.com/mapfiles/place_api/ic...,Villas Victoria Confort Jarretaderas,{'open_now': True},NaN,ChIJdSKPg3xHIYQRlZ0pmfgVPiQ,"{'compound_code': 'MPRG+R4 Jarretaderas, Nayar...",5.0,ChIJdSKPg3xHIYQRlZ0pmfgVPiQ,...,Villas Victoria Confort Jarretaderas,5.0,"[{'author_name': 'anahi chavez', 'author_url':...",1.0,https://www.villasvictoriacomfort.com/,[5],"['It was more than I expected! Cheap, spacious...",NaN,20.692073,-105.274730
2,OPERATIONAL,"{'lat': 20.6939068, 'lng': -105.2763239}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Posada Alejandra,NaN,"[{'height': 349, 'html_attributions': ['<a hre...",ChIJB2Yha4NGIYQRE-GTtF_VSAU,"{'compound_code': 'MPVF+HF Jarretaderas, Nayar...",4.0,ChIJB2Yha4NGIYQRE-GTtF_VSAU,...,Hotel Posada Alejandra,4.0,"[{'author_name': 'Juan Carlos Lopez', 'author_...",54.0,https://www.google.com/url?sa=t&source=web&rct...,"[3, 4, 3, 3, 5]","[""Just for one night it's alright"", 'Discreet'...",NaN,20.693907,-105.276324
3,OPERATIONAL,"{'lat': 20.6241471, 'lng': -105.2243218}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Azul Romero,{'open_now': True},"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJFW5JHF5FIYQRogsYTW-KvMI,"{'compound_code': 'JQFG+M7 Puerto Vallarta, Ja...",3.9,ChIJFW5JHF5FIYQRogsYTW-KvMI,...,Hotel Azul Romero,3.9,"[{'author_name': 'Al Reust', 'author_url': 'ht...",11.0,http://www.azulromero.com/,"[3, 4, 5, 4, 1]","['This is interesting, as it appears to be be ...",NaN,20.624147,-105.224322
4,OPERATIONAL,"{'lat': 20.6225929, 'lng': -105.2232518}",https://maps.gstatic.com/mapfiles/place_api/ic...,Vacational Apartment Apartamento Vacacional De...,NaN,"[{'height': 1052, 'html_attributions': ['<a hr...",ChIJ9321RV5FIYQRwnf0qJkeIqU,"{'compound_code': 'JQFG+2M Puerto Vallarta, Ja...",4.8,ChIJ9321RV5FIYQRwnf0qJkeIqU,...,Vacational Apartment Apartamento Vacacional De...,4.8,"[{'author_name': 'Daniel Chilldres', 'author_u...",11.0,https://www.airbnb.mx/rooms/23351610,"[5, 4, 5, 5, 5]","['I loved it here!', 'The place is clean, plea...",NaN,20.622593,-105.223252


In [3]:
df.iloc[6].all_reviews

"['Aight', 'Bad attitude of service offensive discrimination without giving reason or motive I definitely no longer recommend it', 'The net is very bad to rest because the man who takes care of the television at night makes the television very loud and does not let you sleep all morning.', 'An inexpensive place does not have the required amenities but is passable', 'Something good']"

In [4]:
remove_n = lambda x: re.sub('\\n',' ',x)
remove_singlen = lambda x: re.sub('\n',' ',x)
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

df['all_reviews'] = df.all_reviews.map(remove_n).map(punc_lower).map(alphanumeric)

In [5]:
df.iloc[3].all_reviews

'  this is interesting  as it appears to be be under new ownership  it has a new paint job on the front  i have not taken time to speak with the new managers  in the past few years i know of people who have did extended vacations as the prices were reasonable  access in the local community offers many things  i live across the street    there is a bus route to centro up the hill and return from centro down the hill  they have a few rooms for family or long term  i have seen buses unload and many people take over the hotel in the past  time will tell     comfortable and decent  it is   minutes from the beach nthe water heater does not work and neither does the internet  but good for spending the night    very good cows in guadalajara    very good    change of administration and they did not make the deposit valid according to why the past administration did not give it to them 😒 i had no other choice than to stay there since according to what i had booked and i had about   people obviou

In [6]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [7]:
remove_emoji = lambda x: emoji_pattern.sub(r'', x)

df['all_reviews'] = df.all_reviews.map(remove_emoji)

In [8]:
df.iloc[3].all_reviews

'  this is interesting  as it appears to be be under new ownership  it has a new paint job on the front  i have not taken time to speak with the new managers  in the past few years i know of people who have did extended vacations as the prices were reasonable  access in the local community offers many things  i live across the street    there is a bus route to centro up the hill and return from centro down the hill  they have a few rooms for family or long term  i have seen buses unload and many people take over the hotel in the past  time will tell     comfortable and decent  it is   minutes from the beach nthe water heater does not work and neither does the internet  but good for spending the night    very good cows in guadalajara    very good    change of administration and they did not make the deposit valid according to why the past administration did not give it to them  i had no other choice than to stay there since according to what i had booked and i had about   people obvious

In [9]:
df.iloc[28].all_reviews

'  jungle between the city  very exuberant place          '

In [10]:
df.iloc[14].all_reviews

'  good service and comfortable price                  '

In [11]:
# remove excessive spaces:
remove_spaces = lambda x: re.sub('        ',' ',x)
remove_spaces2 = lambda x: re.sub('   ', ' ', x)

df['all_reviews'] = df.all_reviews.map(remove_spaces).map(remove_spaces2)

In [12]:
df.iloc[28].all_reviews

'  jungle between the city  very exuberant place '

In [13]:
df[df['all_reviews'].str.len() <= 80]

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
12,OPERATIONAL,"{'lat': 20.6314507, 'lng': -105.2229678}",https://maps.gstatic.com/mapfiles/place_api/ic...,plaza central sur 150,NaN,NaN,ChIJzf0oz2NFIYQRuvlEAs6g2Fg,"{'compound_code': 'JQJG+HR Puerto Vallarta, Ja...",5.0,ChIJzf0oz2NFIYQRuvlEAs6g2Fg,...,plaza central sur 150,5.0,"[{'author_name': 'Sebastian Yañez', 'author_ur...",1.0,NaN,[5],quiet humble and very safe neighborhood,NaN,20.631451,-105.222968
14,OPERATIONAL,"{'lat': 20.6336142, 'lng': -105.2261243}",https://maps.gstatic.com/mapfiles/place_api/ic...,Apartamentos Lisboa,{'open_now': False},"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJT--Qj2RFIYQRCkuXi7C7zKI,"{'compound_code': 'JQMF+CH Puerto Vallarta, Ja...",5.0,ChIJT--Qj2RFIYQRCkuXi7C7zKI,...,Apartamentos Lisboa,5.0,"[{'author_name': 'Joseluis Avila', 'author_url...",6.0,NaN,"[5, 5, 5, 5, 5]",good service and comfortable price,NaN,20.633614,-105.226124
16,OPERATIONAL,"{'lat': 20.6387746, 'lng': -105.22502}",https://maps.gstatic.com/mapfiles/place_api/ic...,Depa vallarta,NaN,NaN,ChIJGeUor59FIYQRJ4VgrqfrkuU,"{'compound_code': 'JQQF+GX Puerto Vallarta, Ja...",5.0,ChIJGeUor59FIYQRJ4VgrqfrkuU,...,Depa vallarta,5.0,"[{'author_name': 'Mr Vallarta', 'author_url': ...",1.0,NaN,[5],without fear of life,NaN,20.638775,-105.225020
25,OPERATIONAL,"{'lat': 20.6550001, 'lng': -105.2198112}",https://maps.gstatic.com/mapfiles/place_api/ic...,Petra Glass Hostal,NaN,"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJMSEIsB1PIYQRSGnL5x7Bxi8,"{'compound_code': 'MQ4J+23 Puerto Vallarta, Ja...",5.0,ChIJMSEIsB1PIYQRSGnL5x7Bxi8,...,Petra Glass Hostal,5.0,"[{'author_name': 'Petra Glass', 'author_url': ...",1.0,NaN,[5],very clean and inexpensive,NaN,20.655000,-105.219811
28,OPERATIONAL,"{'lat': 20.6538149, 'lng': -105.2254394}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Río Cuale,NaN,NaN,ChIJZyLH6opFIYQRgw11fW6Zndw,"{'compound_code': 'MQ3F+GR Puerto Vallarta, Ja...",5.0,ChIJZyLH6opFIYQRgw11fW6Zndw,...,Hotel Río Cuale,5.0,"[{'author_name': 'Fernando Martiñon', 'author_...",2.0,NaN,"[5, 5]",jungle between the city very exuberant place,NaN,20.653815,-105.225439
29,OPERATIONAL,"{'lat': 20.6533935, 'lng': -105.2253539}",https://maps.gstatic.com/mapfiles/place_api/ic...,Nora,NaN,"[{'height': 632, 'html_attributions': ['<a hre...",ChIJVVVVVYFFIYQR5ZAO5E7UWkM,"{'compound_code': 'MQ3F+9V Puerto Vallarta, Ja...",4.2,ChIJVVVVVYFFIYQR5ZAO5E7UWkM,...,Nora,4.2,"[{'author_name': 'Jesus Reyna', 'author_url': ...",5.0,NaN,"[5, 5, 5, 1, 5]",ermozo place i loved it i love this place ...,NaN,20.653393,-105.225354
31,OPERATIONAL,"{'lat': 20.65340699999999, 'lng': -105.2253316}",https://maps.gstatic.com/mapfiles/place_api/ic...,Puerto Vallarta,NaN,"[{'height': 1280, 'html_attributions': ['<a hr...",ChIJJ7hW4RNFIYQReKCg-JvCd5w,"{'compound_code': 'MQ3F+9V Puerto Vallarta, Ja...",5.0,ChIJJ7hW4RNFIYQReKCg-JvCd5w,...,Puerto Vallarta,5.0,"[{'author_name': 'IXOTECUTLI ZIGEINER', 'autho...",1.0,NaN,[5],cool place pal shitter,NaN,20.653407,-105.225332
33,OPERATIONAL,"{'lat': 20.6597267, 'lng': -105.22686}",https://maps.gstatic.com/mapfiles/place_api/ic...,Iris,NaN,"[{'height': 2560, 'html_attributions': ['<a hr...",ChIJf23sZ49FIYQRE5ppLkSSQpE,"{'compound_code': 'MQ5F+V7 Puerto Vallarta, Ja...",4.7,ChIJf23sZ49FIYQRE5ppLkSSQpE,...,Iris,4.7,"[{'author_name': 'ANTONIO VILLALVAZO', 'author...",3.0,NaN,"[5, 5, 4]",super good,NaN,20.659727,-105.226860
46,OPERATIONAL,"{'lat': 20.6658674, 'lng': -105.2525124}",https://maps.gstatic.com/mapfiles/place_api/ic...,Condominio Suite Resort,NaN,"[{'height': 1944, 'html_attributions': ['<a hr...",ChIJ0WlxTrhFIYQRoKWZr6nJeaY,"{'compound_code': 'MP8W+8X Puerto Vallarta, Ja...",4.4,ChIJ0WlxTrhFIYQRoKWZr6nJeaY,...,Condominio Suite Resort,4.4,"[{'author_name': 'Rocio Benitez', 'author_url'...",5.0,NaN,"[5, 5, 3, 4, 5]",

In [14]:
df['all_reviews']

0        an excellent place has all the comforts  it ...
1        it was more than i expected  cheap  spacious...
2        just for one night it s alright  discreet  a...
3        this is interesting  as it appears to be be ...
4        i loved it here   the place is clean  pleasa...
                             ...                        
382      love this resort and hotel  best room ever  ...
383                                   really nice view  
384      new accommodations  but landscaping needs to...
385                                                cool 
386                   there was no beef the day i went  
Name: all_reviews, Length: 387, dtype: object

In [15]:
df.iloc[344].all_reviews

'  ok   '

In [16]:
df1 = df[df['all_reviews'].str.len() > 80]

In [17]:
df1.shape

(346, 26)

In [18]:
df1 = df1.reset_index(drop=True)
df1.tail()

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
341,OPERATIONAL,"{'lat': 20.6778627, 'lng': -105.2852463}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Tower 1 at Vidanta Nuevo Vallarta,{'open_now': True},"[{'height': 768, 'html_attributions': ['<a hre...",ChIJ4SJARyNEIYQRx5laF9dUiN0,"{'compound_code': 'MPH7+4W Nuevo Vallarta, Nay...",4.8,ChIJ4SJARyNEIYQRx5laF9dUiN0,...,Grand Luxxe Tower 1 at Vidanta Nuevo Vallarta,4.8,"[{'author_name': 'Diane Tillman', 'author_url'...",371.0,NaN,"[5, 5, 3, 5, 5]",great location to pools bar restaurants m...,NaN,20.677863,-105.285246
342,OPERATIONAL,"{'lat': 20.6851287, 'lng': -105.287574}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grupo Vidanta Headquarters,NaN,"[{'height': 1001, 'html_attributions': ['<a hr...",ChIJK_fqDKNBIYQRU-pYVJ--ZNI,"{'compound_code': 'MPP6+3X Nuevo Vallarta, Nay...",3.8,ChIJK_fqDKNBIYQRU-pYVJ--ZNI,...,Grupo Vidanta Headquarters,3.8,"[{'author_name': 'Elsy B', 'author_url': 'http...",29.0,https://www.vidanta.com/,"[1, 5, 5, 2, 5]",very disappointed it seems that every time ...,NaN,20.685129,-105.287574
343,OPERATIONAL,"{'lat': 20.6828908, 'lng': -105.2788986}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Tower 5,NaN,"[{'height': 2160, 'html_attributions': ['<a hr...",ChIJI9D-5CVEIYQRyClZqRFcuoM,"{'compound_code': 'MPMC+5C Nuevo Vallarta, Nay...",4.8,ChIJI9D-5CVEIYQRyClZqRFcuoM,...,Grand Luxxe Tower 5,4.8,"[{'author_name': 'Mary Mackey', 'author_url': ...",437.0,https://www.vidanta.com/web/nuevo-vallarta,"[5, 5, 5, 5, 5]",the rooms are opulent the roof top pool and...,NaN,20.682891,-105.278899
344,OPERATIONAL,"{'lat': 20.6769383, 'lng': -105.285772}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Punta Tower at Vidanta Nuevo Vallarta,{'open_now': True},"[{'height': 3480, 'html_attributions': ['<a hr...",ChIJ7zhNNCNEIYQRDyRrWYr8hLk,"{'compound_code': 'MPG7+QM Nuevo Vallarta, Nay...",4.9,ChIJ7zhNNCNEIYQRDyRrWYr8hLk,...,Grand Luxxe Punta Tower at Vidanta Nuevo Vallarta,4.9,"[{'author_name': 'John Bittle', 'author_url': ...",64.0,https://www.vidanta.com/en/web/nuevo-vallarta/...,"[5, 5, 5, 5, 5]",love this resort and hotel best room ever ...,NaN,20.676938,-105.285772
345,OPERATIONAL,"{'lat': 20.6843008, 'lng': -105.2780858}",https://maps.gstatic.com/mapfiles/place_api/ic...,Torre 6 The Grand Luxxe,NaN,"[{'height': 2992, 'html_attributions': ['<a hr...",ChIJo9we6EJFIYQRx9y4RoVO4Bs,"{'compound_code': 'MPMC+PQ Jarretaderas, Nayar...",4.6,ChIJo9we6EJFIYQRx9y4RoVO4Bs,...,Torre 6 The Grand Luxxe,4.6,"[{'author_name': 'Dana Nyborg', 'author_url': ...",15.0,NaN,"[4, 5, 3, 5, 5]",new accommodations but landscaping needs to...,NaN,20.684301,-105.278086


In [19]:
df1.describe(include='all')

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
count,346,346,346,346,176,336,346,345,346.000000,346,...,346,346.000000,346,346.000000,256,346,346,1.0,346.000000,346.000000
unique,1,345,4,343,2,336,346,338,NaN,346,...,343,NaN,346,NaN,250,193,346,NaN,NaN,NaN
top,OPERATIONAL,"{'lat': 20.65340699999999, 'lng': -105.2253316}",https://maps.gstatic.com/mapfiles/place_api/ic...,Villa Azul Celeste,{'open_now': True},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJJb0S2URFIYQRRDb2i0fNII0,"{'compound_code': 'MQ3F+9V Puerto Vallarta, Ja...",NaN,ChIJJb0S2URFIYQRRDb2i0fNII0,...,Villa Azul Celeste,NaN,"[{'author_name': 'Juan Manuel Pelayo Santana',...",NaN,https://www.vidanta.com/web/nuevo-vallarta,"[5, 5, 5, 5, 5]",typical passing hotel nthe bad frequented b...,NaN,NaN,NaN
freq,346,2,324,2,136,1,1,2,NaN,1,...,2,NaN,1,NaN,3,65,1,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.389595,NaN,...,NaN,4.388439,NaN,412.187861,NaN,NaN,NaN,2.0,20.620304,-105.241177
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.499050,NaN,...,NaN,0.498443,NaN,899.480495,NaN,NaN,NaN,NaN,0.034150,0.016527
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,...,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,2.0,20.532667,-105.291951
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.200000,NaN,...,NaN,4.200000,NaN,11.000000,NaN,NaN,NaN,2.0,20.600183,-105.246052
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,NaN,...,NaN,4.500000,NaN,46.000000,NaN,NaN,NaN,2.0,20.614583,-105.236044
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.700000,NaN,...,NaN,4.700000,NaN,286.000000,NaN,NaN,NaN,2.0,20.643411,-105.231297


In [20]:
df1[df1['rating_x']<=3]

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
57,OPERATIONAL,"{'lat': 20.6590553, 'lng': -105.2469}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Villa Varadero,NaN,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJOdpzlbxFIYQRI-mzY60HRcQ,"{'compound_code': 'MQ53+J6 Puerto Vallarta, Ja...",2.7,ChIJOdpzlbxFIYQRI-mzY60HRcQ,...,Hotel Villa Varadero,2.7,"[{'author_name': 'xaviere pedroxa lopex', 'aut...",3.0,NaN,"[1, 4, 3]",there is an error a maps guides you to a v...,NaN,20.659055,-105.246900
130,OPERATIONAL,"{'lat': 20.6263699, 'lng': -105.2281492}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Vallarta,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJ093DB2dFIYQRYxq91vgbkJw,"{'compound_code': 'JQGC+GP Puerto Vallarta, Ja...",2.6,ChIJ093DB2dFIYQRYxq91vgbkJw,...,Hotel Vallarta,2.6,"[{'author_name': 'Kimberly Tyner', 'author_url...",7.0,NaN,"[1, 3, 1, 3, 4]",cruel people good place to rest only that t...,NaN,20.626370,-105.228149
142,OPERATIONAL,"{'lat': 20.613498, 'lng': -105.2328524}",https://maps.gstatic.com/mapfiles/place_api/ic...,The hostal spot,NaN,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJ28z2UUVFIYQRiW3MK5XgGnc,"{'compound_code': 'JQ78+9V Puerto Vallarta, Ja...",2.6,ChIJ28z2UUVFIYQRiW3MK5XgGnc,...,The hostal spot,2.6,"[{'author_name': 'Karen gudgel', 'author_url':...",7.0,NaN,"[1, 5, 5, 1, 1]",no hot water and the electricity was only a ...,NaN,20.613498,-105.232852
158,OPERATIONAL,"{'lat': 20.6131722, 'lng': -105.232317}",https://maps.gstatic.com/mapfiles/place_api/ic...,Departamento en el corazón de Vallarta,NaN,"[{'height': 2592, 'html_attributions': ['<a hr...",ChIJA7LxQ0VFIYQRiylo05bpyrM,"{'compound_code': 'JQ79+73 Puerto Vallarta, Ja...",1.0,ChIJA7LxQ0VFIYQRiylo05bpyrM,...,Departamento en el corazón de Vallarta,1.0,"[{'author_name': 'Addy Munguia', 'author_url':...",2.0,NaN,"[1, 1]",the business has been closed for years and g...,NaN,20.613172,-105.232317
279,OPERATIONAL,"{'lat': 20.5916992, 'lng': -105.24157}",https://maps.gstatic.com/mapfiles/place_api/ic...,Casa Destination Of Health Resort,NaN,"[{'height': 4128, 'html_attributions': ['<a hr...",ChIJrZeSN7ZaIYQRvhiC-HiH6lw,"{'compound_code': 'HQR5+M9 Puerto Vallarta, Ja...",1.8,ChIJrZeSN7ZaIYQRvhiC-HiH6lw,...,Casa Destination Of Health Resort,1.8,"[{'author_name': 'Bianca Ferratt', 'author_url...",4.0,http://www.casadestinationofhealth.mxhotel.site/,"[1, 1, 4, 1]",i made a reservation and the address does no...,NaN,20.591699,-105.241570
313,OPERATIONAL,"{'lat': 20.5483213, 'lng': -105.2731479}",https://maps.gstatic.com/mapfiles/place_api/ic...,Gran park royal,NaN,"[{'height': 943, 'html_attributions': ['<a hre...",ChIJVSghLVhbIYQRd69si_Ia1-U,"{'compound_code': 'GPXG+8P Aguacate, Jalisco, ...",2.7,ChIJVSghLVhbIYQRd69si_Ia1-U,...,Gran park royal,2.7,"[{'author_name': 'Kevin MX', 'author_url': 'ht...",3.0,NaN,"[4, 1, 3]",nice hotel good food pool is cold kids po...,NaN,20.548321,-105.273148


In [ ]:
# df1.to_csv(r'hotels_cleaned.csv', index=True, header=True, index_label=False)

In [21]:
df1 = df1[df1['rating_x']>3.5]

In [22]:
df1.shape

(329, 26)

# Word Embeddings

In [23]:
df1.iloc[0].all_reviews

'  an excellent place has all the comforts  it is a wonderful place and its people are very friendly   '

In [ ]:
def w2v_tokenizer(document):
    stoplist = set('for a an it its very of the and to in'.split())
    texts = [word for word in document.lower().split() if word not in stoplist]
    
    return texts

In [ ]:
df1.tail()

In [ ]:
df2 = df1.all_reviews.apply(w2v_tokenizer)

In [ ]:
df2

In [ ]:
w2v_tokenizer(df1.iloc[1].all_reviews)

In [ ]:
def viable_words(words):
    viable_words = []
    for word in words:
        try:
            model[word]
            viable_words.append(word)
        except:
            pass
    return viable_words

In [ ]:
df2 = df2.apply(viable_words)

In [ ]:
df2

# Pre-trained Word2Vect Model

In [24]:
import os

# Setup nltk corpora path and Google Word2Vec location
google_vec_file = '/Users/lindsayread/Downloads/GoogleNews-vectors-negative300.bin'

In [25]:
model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [89]:
def word_vectorizer(document):
    stoplist = set('for a an it its very of the and to in'.split())
    texts = [word for word in document.lower().split() if word not in stoplist]
    
    return texts

In [92]:
def viable_words(words):
    viable_words = []
    for word in words:
        try:
            model[f'{word}']
            viable_words.append(word)
        except:
            pass
    return viable_words
            

In [94]:
viables_df = initial_df.apply(viable_words)

In [96]:
first_hotel = viables_df.iloc[0]
first_hotel

['excellent',
 'place',
 'has',
 'all',
 'comforts',
 'is',
 'wonderful',
 'place',
 'people',
 'are',
 'friendly']

In [132]:
first_five = viables_df.iloc[0:5]
first_five

0    [excellent, place, has, all, comforts, is, won...
1    [was, more, than, i, expected, cheap, spacious...
2    [just, one, night, s, alright, discreet, simpl...
3    [this, is, interesting, as, appears, be, be, u...
4    [i, loved, here, place, is, clean, pleasant, e...
Name: all_reviews, dtype: object

In [138]:
def mean_hotel_vector(hotel):

    df_first_word = pd.DataFrame([model[hotel[0]]])
    for word in df_first_word[1:]:
        df_first_word = pd.concat([df_first_word, pd.DataFrame([model[f'{word}']])], ignore_index=True)
    first_hotel = pd.DataFrame(df_first_word.mean())
    return(first_hotel)

In [153]:
df_all_vectors = df_vector_first
for hotel in first_five:
    first_word = pd.DataFrame([model[hotel[0]]])
    for word in hotel[1:]:
        first_word = pd.concat([df_first, pd.DataFrame([model[f'{word}']])], ignore_index=True)
    first_word = pd.DataFrame(first_word.mean())
    df_all_vectors = pd.concat([df_all_vectors, first_word], axis=1, ignore_index=True)
    #print(hotel[1])
df_all_vectors

,0,1,2,3,4,5
0,-0.020985,-0.018016,-0.015859,-0.010447,-0.005076,0.006317
1,0.079720,0.102048,0.065061,0.070473,0.074267,0.078773
2,0.014596,0.016960,0.025586,0.016329,0.009595,0.000033
3,0.039373,0.037384,0.056274,0.050008,0.043132,0.046061
4,-0.042453,-0.054784,-0.036535,-0.037847,-0.037923,-0.039462
...,...,...,...,...,...,...
295,0.057663,0.072959,0.048016,0.063844,0.058392,0.055065
296,0.041021,0.053716,0.023931,0.038671,0.031214,0.048589
297,-0.022186,-0.036939,-0.044263,-0.024874,-0.016879,-0.021670
298,0.039479,0.042252,0.042781,0.055557,0.027847,0.018122


In [177]:
def mapped_vectors(viable_words):
    df_first = pd.DataFrame([model[viable_words[0]]])
    for word in viable_words[1:]:
        df_first = pd.concat([df_first, pd.DataFrame([model[f'{word}']])], ignore_index=True)
    df_mean = pd.DataFrame(df_first.mean())
    return df_mean.T
        

In [178]:
second_V = mapped_vectors(second_hotel)
second_V

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.019035,0.029907,0.059304,0.10355,0.005379,0.023773,0.008163,-0.111519,0.104652,0.082924,...,-0.104172,0.053316,-0.093506,0.020936,0.036129,-0.052326,-0.004178,-0.034481,0.027336,-0.041651


In [185]:
third_hotel = viables_df.iloc[2]

In [187]:
third_V = mapped_vectors(third_hotel)
third_V

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.017837,0.03663,-0.004139,0.061884,-0.056658,0.002148,0.03781,-0.086593,0.089868,0.102893,...,-0.061085,0.071887,-0.08144,0.037477,-0.026248,0.00987,0.001176,-0.022683,0.005633,-0.028478


In [181]:
for hotel in first_five:
    second_V = pd.concat([second_V, mapped_vectors(hotel)])
second_V

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.019035,0.029907,0.059304,0.103550,0.005379,0.023773,0.008163,-0.111519,0.104652,0.082924,...,-0.104172,0.053316,-0.093506,0.020936,0.036129,-0.052326,-0.004178,-0.034481,0.027336,-0.041651
0,-0.020985,0.079720,0.014596,0.039373,-0.042453,0.007990,0.044106,-0.109730,0.002894,0.106301,...,-0.139549,-0.022568,-0.140958,-0.000347,-0.032293,0.057663,0.041021,-0.022186,0.039479,0.017561
0,-0.019035,0.029907,0.059304,0.103550,0.005379,0.023773,0.008163,-0.111519,0.104652,0.082924,...,-0.104172,0.053316,-0.093506,0.020936,0.036129,-0.052326,-0.004178,-0.034481,0.027336,-0.041651
0,0.017837,0.036630,-0.004139,0.061884,-0.056658,0.002148,0.037810,-0.086593,0.089868,0.102893,...,-0.061085,0.071887,-0.081440,0.037477,-0.026248,0.009870,0.001176,-0.022683,0.005633,-0.028478
0,0.013963,0.004875,0.019757,0.093845,-0.042801,-0.014613,0.018288,-0.080157,0.095798,0.067508,...,-0.061187,0.057206,-0.080036,0.038635,-0.006875,0.017680,-0.004359,-0.033175,0.026842,-0.050020
0,-0.003060,0.020569,0.042245,0.044713,-0.046801,0.011214,0.026570,-0.080710,0.059676,0.059297,...,-0.110012,0.028893,-0.101961,0.053882,-0.002300,0.041499,-0.004092,-0.040666,0.030417,-0.018074


In [188]:
last_hotel = viables_df.iloc[-1]

In [189]:
mapped_vectors(last_hotel)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.00751,0.038378,0.022828,0.082442,-0.036694,-0.014666,0.067579,-0.066124,0.043483,0.057335,...,-0.094858,0.02564,-0.094372,0.017017,-0.00477,0.025147,0.033896,-0.076249,0.072913,-0.021889


In [183]:
pd.DataFrame(cosine_similarity(second_V))

,0,1,2,3,4,5
0,1.000000,0.590262,1.000000,0.725514,0.711866,0.717221
1,0.590262,1.000000,0.590262,0.760175,0.696436,0.839844
2,1.000000,0.590262,1.000000,0.725514,0.711866,0.717221
3,0.725514,0.760175,0.725514,1.000000,0.840045,0.892312
4,0.711866,0.696436,0.711866,0.840045,1.000000,0.860659
5,0.717221,0.839844,0.717221,0.892312,0.860659,1.000000


In [179]:
mean_vector = lambda x: mapped_vectors(x)

df_v_five = pd.DataFrame(first_five.map(mean_vector))

In [180]:
df_v_five

,all_reviews
0,0 1 2 3 ...
1,0 1 2 3 ...
2,0 1 2 3 ...
3,0 1 2 3 ...
4,0 1 2 3 ...


In [162]:
second_hotel = viables_df.iloc[1]
second_hotel

['was',
 'more',
 'than',
 'i',
 'expected',
 'cheap',
 'spacious',
 'beautiful',
 'rooms',
 'with',
 'retro',
 'television',
 'air',
 'conditioning',
 'i',
 'loved']

In [163]:
df_second = pd.DataFrame([model[second_hotel[0]]])
for word in second_hotel[1:]:
    df_second = pd.concat([df_second, pd.DataFrame([model[f'{word}']])], ignore_index=True)
df_second

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.026001,-0.001892,0.185547,-0.051758,0.005127,-0.109863,-0.008179,-0.088379,0.096680,0.048340,...,-0.300781,-0.045898,-0.174805,0.233398,0.022583,0.110352,-0.103516,-0.121582,0.221680,-0.021973
1,0.063965,-0.255859,-0.084473,0.151367,-0.081543,0.038818,-0.036377,-0.117188,0.041992,0.001595,...,0.017578,0.095215,0.039307,0.169922,-0.106445,-0.049072,0.123047,0.027466,0.064941,0.062012
2,-0.076660,-0.104004,-0.001755,0.209961,0.090820,0.019287,0.044434,-0.076660,0.148438,0.160156,...,-0.106934,0.125977,-0.063965,0.245117,-0.045898,0.104004,0.049561,-0.019653,-0.034424,0.000751
3,-0.225586,-0.019531,0.090820,0.237305,-0.029297,0.093262,-0.058838,-0.041016,0.052246,0.020020,...,0.075684,0.225586,-0.111328,-0.039551,0.035156,-0.113281,-0.176758,0.028198,-0.177734,-0.006042
4,0.063477,0.265625,0.069824,-0.008728,-0.175781,0.012573,0.184570,-0.037842,0.208984,0.087891,...,-0.104980,-0.125977,-0.080078,0.007141,0.172852,0.215820,-0.019775,-0.096680,0.154297,-0.216797
5,0.067383,-0.081055,-0.103027,0.253906,-0.156250,0.167969,0.099121,-0.086426,0.294922,0.232422,...,-0.044189,0.176758,-0.153320,0.187500,-0.103027,0.137695,0.103027,-0.166016,-0.079590,0.072266
6,-0.018799,0.185547,-0.062256,-0.114746,0.136719,-0.202148,-0.194336,0.014526,0.178711,0.207031,...,-0.335938,0.113770,-0.202148,-0.054688,0.165039,-0.080078,0.202148,-0.120117,0.028198,0.024414
7,-0.018311,0.055664,-0.011536,0.072754,0.151367,-0.061768,0.206055,-0.153320,-0.059082,0.228516,...,-0.097656,0.096191,-0.099609,-0.002853,-0.036377,0.154297,0.061523,-0.345703,0.110840,0.033447
8,0.030762,0.214844,0.232422,0.088867,-0.043701,0.140625,-0.073242,0.025146,0.239258,-0.120117,...,-0.137695,0.025513,0.041260,0.115723,0.127930,-0.542969,0.118164,0.162109,0.135742,0.200195
9,-0.024902,0.021973,-0.035400,0.136719,0.016357,-0.027954,-0.024292,-0.139648,0.057373,0.062256,...,-0.020752,0.003082,-0.138672,-0.024414,-0.093262,-0.047607,-0.075684,0.010803,-0.018799,-0.068848


In [165]:
pd.Series(df_second.mean())

0     -0.019035
1      0.029907
2      0.059304
3      0.103550
4      0.005379
         ...   
295   -0.052326
296   -0.004178
297   -0.034481
298    0.027336
299   -0.041651
Length: 300, dtype: float64

In [152]:
df_vector_first

0     -0.020985
1      0.079720
2      0.014596
3      0.039373
4     -0.042453
         ...   
295    0.057663
296    0.041021
297   -0.022186
298    0.039479
299    0.017561
Length: 300, dtype: float64

In [101]:
df_first = pd.DataFrame([model[first_hotel[0]]])
for word in first_hotel[1:]:
    df_first = pd.concat([df_first, pd.DataFrame([model[f'{word}']])], ignore_index=True)
df_first

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.212891,-0.004303,-0.180664,-0.007568,0.112793,0.163086,-0.014709,-0.078613,-0.164062,0.279297,...,0.136719,0.000282,-0.173828,0.004242,-0.081055,0.013550,-0.008362,-0.129883,-0.215820,0.012268
1,-0.207031,0.203125,0.041992,-0.115723,-0.150391,-0.243164,0.134766,-0.164062,0.151367,0.105469,...,-0.198242,0.026367,-0.166016,-0.082031,-0.152344,-0.052002,0.010681,0.138672,0.036865,-0.028687
2,-0.044922,-0.030396,0.002350,-0.006592,0.087891,0.032959,-0.094238,-0.241211,-0.057373,0.226562,...,-0.289062,-0.058594,-0.109863,0.190430,0.069824,0.018311,0.027588,0.012268,-0.088379,-0.015625
3,-0.007812,-0.027954,0.040527,0.117676,-0.013306,-0.090820,0.023682,-0.024048,0.138672,0.079590,...,-0.079590,0.043945,-0.043457,-0.085449,-0.106934,0.044434,0.123047,0.039551,-0.060547,0.008057
4,0.253906,0.285156,-0.029907,0.113770,0.120605,0.250000,0.168945,-0.363281,-0.147461,0.132812,...,-0.375000,-0.168945,-0.195312,-0.135742,-0.036621,0.003464,0.006348,-0.007660,0.066406,0.223633
5,0.007050,-0.073242,0.171875,0.022583,-0.132812,0.198242,0.112793,-0.107910,0.071777,0.020874,...,-0.233398,-0.036377,-0.093750,0.182617,0.027100,0.127930,-0.024780,0.011230,0.164062,0.106934
6,0.006256,0.019287,-0.089844,0.142578,0.102051,-0.135742,0.138672,-0.071777,-0.125977,0.163086,...,-0.113281,0.023682,-0.226562,-0.063965,-0.087402,-0.006500,0.147461,-0.161133,0.001160,-0.106445
7,-0.207031,0.203125,0.041992,-0.115723,-0.150391,-0.243164,0.134766,-0.164062,0.151367,0.105469,...,-0.198242,0.026367,-0.166016,-0.082031,-0.152344,-0.052002,0.010681,0.138672,0.036865,-0.028687
8,0.263672,-0.019165,0.028931,0.234375,-0.007874,0.016357,-0.112793,0.066406,0.001556,-0.040283,...,0.020386,0.131836,-0.255859,-0.098145,0.068848,0.087891,0.021118,-0.066895,0.207031,-0.138672
9,-0.096680,-0.026367,0.090332,0.032227,-0.245117,0.056152,0.025269,-0.009399,0.054932,0.061279,...,0.024170,-0.123047,0.031494,0.039307,0.032227,0.208008,-0.055908,-0.019653,0.213867,0.136719


In [150]:
df_vector_first = pd.Series(df_first.mean())
df_vector_first

0     -0.020985
1      0.079720
2      0.014596
3      0.039373
4     -0.042453
         ...   
295    0.057663
296    0.041021
297   -0.022186
298    0.039479
299    0.017561
Length: 300, dtype: float64

In [143]:
for hotel in first_five.iloc[1:]:
    for word in hotel:
        print(word)

was
more
than
i
expected
cheap
spacious
beautiful
rooms
with
retro
television
air
conditioning
i
loved
just
one
night
s
alright
discreet
simple
clean
place
with
excellent
service
cheap
but
does
not
have
air
conditioning
hot
season
is
almost
impossible
rest
they
have
cheap
laundry
service
good
location
with
secure
parking
quiet
place
clean
comfortable
inexpensive
place
ideal
spend
night
rest
this
is
interesting
as
appears
be
be
under
new
ownership
has
new
paint
job
on
front
i
have
not
taken
time
speak
with
new
managers
past
few
years
i
know
people
who
have
did
extended
vacations
as
prices
were
reasonable
access
local
community
offers
many
things
i
live
across
street
there
is
bus
route
centro
up
hill
return
from
centro
down
hill
they
have
few
rooms
family
or
long
term
i
have
seen
buses
unload
many
people
take
over
hotel
past
time
will
tell
comfortable
decent
is
minutes
from
beach
nthe
water
heater
does
not
work
neither
does
internet
but
good
spending
night
good
cows
good
change
administr

In [ ]:
remove_n = lambda x: re.sub('\\n',' ',x)

df['all_reviews'] = df.all_reviews.map(remove_n)

In [88]:
mini_initial_df.apply(print_not_viable)

not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable
not viable

0    None
1    None
2    None
3    None
4    None
Name: all_reviews, dtype: object

In [85]:
initial_df = df1.all_reviews.apply(word_vectorizer)
initial_df.head()

0    [excellent, place, has, all, comforts, is, won...
1    [was, more, than, i, expected, cheap, spacious...
2    [just, one, night, s, alright, discreet, simpl...
3    [this, is, interesting, as, appears, be, be, u...
4    [i, loved, here, place, is, clean, pleasant, e...
Name: all_reviews, dtype: object

In [71]:
def dimensions(viable_words):
    for word in viable_words:
        df = pd.DataFrame([model[word[0]]])
    
    #for word in viable_words[1:]:
        #df = pd.concat([df, pd.DataFrame([model[word]])], ignore_index=True)
    return df

In [86]:
mini_initial_df = initial_df[0:5]
mini_initial_df

0    [excellent, place, has, all, comforts, is, won...
1    [was, more, than, i, expected, cheap, spacious...
2    [just, one, night, s, alright, discreet, simpl...
3    [this, is, interesting, as, appears, be, be, u...
4    [i, loved, here, place, is, clean, pleasant, e...
Name: all_reviews, dtype: object

In [79]:
mini_initial_df.apply(dimensions)

KeyError: "word '-0.21289062' not in vocabulary"

In [77]:
mini_initial_df.iloc[0]

[array([-2.12890625e-01, -4.30297852e-03, -1.80664062e-01, -7.56835938e-03,
         1.12792969e-01,  1.63085938e-01, -1.47094727e-02, -7.86132812e-02,
        -1.64062500e-01,  2.79296875e-01, -2.22656250e-01, -1.37329102e-02,
         1.45507812e-01, -4.56542969e-02,  5.85937500e-02,  1.85546875e-01,
         1.13769531e-01,  1.13769531e-01, -1.46484375e-01, -1.25122070e-02,
         8.78906250e-02,  1.44531250e-01, -6.88476562e-02,  1.68945312e-01,
         2.02148438e-01,  3.64685059e-03, -1.24023438e-01,  1.76757812e-01,
        -4.24804688e-02, -3.54003906e-03,  4.93164062e-02, -1.74804688e-01,
         2.53906250e-01,  1.32812500e-01,  2.63671875e-01,  2.87109375e-01,
        -1.98242188e-01, -9.81445312e-02, -1.18652344e-01, -1.43554688e-01,
         3.08593750e-01, -8.93554688e-02, -7.17163086e-03,  3.46679688e-02,
        -7.86132812e-02, -1.50390625e-01,  2.15820312e-01, -1.17675781e-01,
         2.85644531e-02, -2.41210938e-01,  1.22558594e-01,  1.28906250e-01,
        -9.2

In [73]:
mini_initial_df.apply(dimensions)

KeyError: "word 'a' not in vocabulary"

In [ ]:
all_vectors = []
for word in df2.iloc[0]:
    all_vectors.append(model[word])

In [ ]:
df2.iloc[0]

In [ ]:
np.mean(all_vectors)

In [ ]:
def hotel_nlp_vector(viable_words):
    all_vectors = []
    for word in viable_words:
        all_vectors.append(model[word])
    return np.mean(all_vectors)

In [ ]:
nlp_vectors = df2.apply(hotel_nlp_vector)

In [ ]:
nlp_vectors

In [ ]:
nlp_vectors.shape

In [ ]:
df1.shape

In [ ]:
df1['w2vVector'] = nlp_vectors

In [ ]:
df1.describe(include='all')

In [ ]:
df1.to_csv(r'nlp_embeddings.csv', index=True, header=True, index_label=False)

In [ ]:
all_vectors = []
for i in df2.iloc[0]:
    all_vectors.append(model[i])
np.mean(all_vectors)

In [ ]:
all_vectors = []
for i in df2.iloc[13]:
    all_vectors.append(model[i])
np.mean(all_vectors)

In [ ]:
df2.iloc[0]

In [ ]:
from nltk.tokenize import TreebankWordTokenizer

In [ ]:
w2v

In [ ]:
def get_embedded_review(review, embedding, tokenizer=TreebankWordTokenizer().tokenize):
    """
    Tokenizes a review and then returns average embedding vector across all the tokens
    
    :param review: <str> text of a single review
    :param embedding: keyed vectors representing word embeddings
    :param tokenizer: function that turns a long string of text into list of tokens
    :return: vector of the same dimension as embedding vectors, calculated as average of word vectors in review
    """
    tokens = tokenizer(review)
    # all text cleaning/pre-processing goes here
    # can put weight to tokens here... if you include "the", this word will have the same weight as all other words
    return np.mean([embedding[tok] for tok in tokens if tok in embedding], axis=0)

In [ ]:
df1.iloc[0].all_reviews

In [ ]:
stop_list = set('for a an it its very of the and to in'.split())
tree_tokens = [df1.all_reviews.apply(TreebankWordTokenizer().tokenize) if word not in stop_list]
tree_tokens.head()

In [ ]:
tree_tokens = df1.all_reviews.apply(TreebankWordTokenizer().tokenize)
tree_tokens.head()

In [ ]:
def no_stops(words):
    stop_list = set('for a an it its very of the and to in'.split())
    return [word for word in words if word not in stop_list]

In [ ]:
cleaned_tokens = tree_tokens.apply(no_stops)

In [ ]:
cleaned_tokens.head()

In [ ]:
def get_embedded_review2(review, embedding):
    """
    Tokenizes a review and then returns average embedding vector across all the tokens
    
    :param review: <str> text of a single review
    :param embedding: keyed vectors representing word embeddings
    :param tokenizer: function that turns a long string of text into list of tokens
    :return: vector of the same dimension as embedding vectors, calculated as average of word vectors in review
    """
    #tokens = tokenizer(review)
    # all text cleaning/pre-processing goes here
    # can put weight to tokens here... if you include "the", this word will have the same weight as all other words
    return np.mean([embedding[tok] for tok in review if tok in embedding], axis=0)

In [ ]:
embedded2 = cleaned_tokens.apply(viable_words)

In [ ]:
embedded2

In [ ]:
vectorized3 = embedded2.apply(hotel_nlp_vector)
vectorized3.head()

In [ ]:
def apply_vectors(list_of_words):
    list_of_vectors = []
    for word in list_of_words:
        list_of_vectors.append(model[word])
    return list_of_vectors

In [ ]:
vectorized = embedded2.apply(apply_vectors)
vectorized.head()

In [ ]:
vectorized.iloc[0]

In [ ]:
embedded2_mean = np.mean(vectorized)
embedded2_mean.head()

In [ ]:
embedded2_mean = np.mean([model[word] for word in embedded2], axis=0)